In [ ]:
# word form -> (morphology, orthography, phonology and multi word expressions)
#       properties of the word and characters themselvies. ie. silent letters, sounds like, pre/suffix
# semantic relations -> (synonyms, polysemy, hypernymy)

# association -> properties of the object
# encyldopedic -> knowledge from everyday life
# combinations 


# could use a tree structure -> recursively find group with remaining words
# solve all at once -> using fact that they all depend on each other (not fall for red herring)

In [ ]:
prompt = """Solve today’s NYT Connections game. Here are the instructions for how to play this game:
Find groups of four items that share something in common.
Category Examples:
FISH: Bass, Flounder, Salmon, Trout
FIRE ___: Ant, Drill, Island, Opal
Categories will always be more specific than
‘5-LETTER-WORDS’, ‘NAMES’, or ‘VERBS.’
Example 1:
Words: [‘DART’, ‘HEM’, ‘PLEAT’, ‘SEAM’,
‘CAN’, ‘CURE’, ‘DRY’, ‘FREEZE’, ‘BITE’,
‘EDGE’, ‘PUNCH’, ‘SPICE’, ‘CONDO’, ‘HAW’,
‘HERO’, ‘LOO’]
Groupings:
1. Things to sew: [‘DART’, ‘HEM’, ‘PLEAT’,
‘SEAM’]
2. Ways to preserve food: [‘CAN’, ‘CURE’,
‘DRY’, ‘FREEZE’]
3. Sharp quality: [‘BITE’, ‘EDGE’, ‘PUNCH’,
‘SPICE’]
4. Birds minus last letter: [‘CONDO’, ‘HAW’,
‘HERO’, ‘LOO’]
Example 2:
Words: [1COLLECTIVE’, ‘COMMON’, ‘JOINT’,
‘MUTUAL’, ‘CLEAR’, ‘DRAIN’, ‘EMPTY’,
‘FLUSH’, ‘CIGARETTE’, ‘PENCIL’, ‘TICKET’,
‘TOE’, ‘AMERICAN’, ‘FEVER’, ‘LUCID’,
‘PIPE’]
Groupings:
1. Shared: [‘COLLECTIVE’, ‘COMMON’,
‘JOINT’, ‘MUTUAL’]
2. Rid of contents: [‘CLEAR’, ‘DRAIN’,
‘EMPTY’, ‘FLUSH’]
3. Associated with “stub”: [‘CIGARETTE’,
‘PENCIL’, ‘TICKET’, ‘TOE’]
4. __ Dream: [‘AMERICAN’, ‘FEVER’, ‘LU-
CID’, ‘PIPE’])
Example 3:
Words: [‘HANGAR’, ‘RUNWAY’, ‘TARMAC’,
‘TERMINAL’, ‘ACTION’, ‘CLAIM’, ‘COM-
PLAINT’, ‘LAWSUIT’, ‘BEANBAG’, ‘CLUB’,
‘RING’, ‘TORCH’, ‘FOXGLOVE’, ‘GUMSHOE’,
‘TURNCOAT’, ‘WINDSOCK’]
Groupings:
1. Parts of an airport: [‘HANGAR’, ‘RUNWAY’,
‘TARMAC’, ‘TERMINAL’]
2. Legal terms: [‘ACTION’, ‘CLAIM’, ‘COM-
PLAINT’, ‘LAWSUIT’]
3. Things a juggler juggles: [‘BEANBAG’,
‘CLUB’, ‘RING’, ‘TORCH’]
4. Words ending in clothing: [‘FOXGLOVE’,
‘GUMSHOE’, ‘TURNCOAT’, ‘WIND-
SOCK’]
Categories share commonalities:
• There are 4 categories of 4 words each
• Every word will be in only 1 category
• One word will never be in two categories
• As the category number increases, the connec-
tions between the words and their category
become more obscure. Category 1 is the most
easy and intuitive and Category 4 is the hard-
est
• There may be a red herrings (words that seems
to belong together but actually are in separate
categories)
• Category 4 often contains compound words
with a common prefix or suffix word
• A few other common categories include word
and letter patterns, pop culture clues (such as
music and movie titles) and fill-in-the-blank
phrases
You will be given a new example (Example 4) with
today’s list of words. First explain your reason
for each category and then give your final answer
following the structure below (Replace Category 1,
2, 3, 4 with their names instead)
Groupings:
Category1: [word1, word2, word3, word4]
Category2: [word5, word6, word7, word8]
Category3: [word9, word10, word11, word12]
Category4: [word13, word14, word15, word16]
Remember that the same word cannot be re-
peated across multiple categories, and you need
to output 4 categories with 4 distinct words each.
Also do not make up words not in the list. This is
the most important rule. Please obey
Example 4:
Words : [InsertGame]
Grouping"""

In [ ]:
class ConnectionsEncoder:
    def __init__():
        self.semantic_encoder = TransformerEncoder(
            embedding_dim = 768,
            num_heads=8,
            include_wordnet=True,
            include_conceptnet=True
        )

        # fine tune on history
        self.pattern = ConnectionsTransformer(
            base_model = 'llama-3-1-8b',
            strategy = 'few_shot'
        )

    def relationships(self, words):
        # broad relationship
        semantic_embed = self.semantic_encoder(words)

        
        patterns = self.pattern_matcher(
        words=words,
        semantic_context=semantic_embeddings,
        prompt="""
        Consider these words and their semantic relationships.
        Look for:
        1. Direct category membership
        2. Metaphorical connections
        3. Word play or double meanings
        4. Context-dependent relationships
        """
    )
    
    return self.combine_evidence(semantic_embeddings, patterns)


In [ ]:
# Phase 1: Semantic Pre-training
semantic_training = {
    "objectives": [
        "word_similarity",
        "category_membership",
        "conceptual_relations"
    ],
    "data_sources": [
        "WordNet",
        "ConceptNet",
        "Word2Vec"
    ]
}

# Phase 2: Connections-specific Fine-tuning
connections_training = {
    "objectives": [
        "group_identification",
        "multiple_meaning_awareness",
        "context_sensitivity"
    ],
    "techniques": [
        "few_shot_learning",
        "contrastive_learning",
        "adversarial_examples"
    ]
}

In [ ]:
class Solver:
    def __init__():
        self.encoder = ConnectionsEncoder() 
        self.model = 'llama3.1*B'

    def solve(self, words):
        